In [1]:
!pip install biopython

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 62.4 MB/s eta 0:00:00


In [2]:
!apt-get update
!apt-get install cd-hit

Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:2 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Hit:5 https://cli.github.com/packages stable InRelease
Get:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Hit:7 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:10 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:11 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [2,065 kB]
Get:12 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:13 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 Packages

In [3]:
from Bio import SeqIO
from pathlib import Path
import subprocess

# KONFIGURASI

In [20]:
RAW_FASTA = "data/raw/PlasmoDB-68_Plasmodium_Isolates.fasta"
FILTERED_FASTA = "data/processed/plasmo_filtered_100.fasta"
NONREDUNDANT_FASTA = "data/processed/plasmo_nonredundant_cdhit.fasta"
CDHIT_CLUSTER = NONREDUNDANT_FASTA + ".clstr"

Path("data/raw").mkdir(parents=True, exist_ok=True)
Path("data/processed").mkdir(parents=True, exist_ok=True)

MIN_LEN = 100   # buang sekuens < 100 aa
CDHIT_IDENTITY = 0.5  # 20% identity
CDHIT_WORDSIZE = 2    # sesuai cutoff rendah

# FILTER SEKUENS PENDEK (<100 AA)

In [21]:
print("Memfilter sekuens pendek (<100 aa)...")
filtered_records = [rec for rec in SeqIO.parse(RAW_FASTA, "fasta") if len(rec.seq) >= MIN_LEN]
SeqIO.write(filtered_records, FILTERED_FASTA, "fasta")
print(f"✅ {len(filtered_records)} sekuens disimpan ke {FILTERED_FASTA}")

Memfilter sekuens pendek (<100 aa)...
✅ 24625 sekuens disimpan ke data/processed/plasmo_filtered_100.fasta


# JALANKAN CD-HIT

In [24]:
print("Menjalankan CD-HIT...")
cmd = [
    "cd-hit",
    "-i", FILTERED_FASTA,
    "-o", NONREDUNDANT_FASTA,
    "-c", str(CDHIT_IDENTITY),
    "-n", str(CDHIT_WORDSIZE),
    "-d", "0"  # simpan header penuh
]
subprocess.run(cmd, check=True)
print(f"✅ CD-HIT selesai. Hasil nonredundan disimpan ke {NONREDUNDANT_FASTA}")
print(f"📂 File cluster detail: {CDHIT_CLUSTER}")

Menjalankan CD-HIT...
✅ CD-HIT selesai. Hasil nonredundan disimpan ke data/processed/plasmo_nonredundant_cdhit.fasta
📂 File cluster detail: data/processed/plasmo_nonredundant_cdhit.fasta.clstr


# OPSIONAL: CETAK RINGKASAN CLUSTER

In [23]:
print("\nContoh isi file cluster:")
with open(CDHIT_CLUSTER) as f:
    for i, line in enumerate(f):
        if i > 30:  # tampilkan 30 baris pertama saja
            break
        print(line.strip())


Contoh isi file cluster:
>Cluster 0
0	478aa, >AB477120... at 51.67%
1	5977aa, >AB444134... at 50.29%
2	5977aa, >AB444136... at 50.29%
3	852aa, >KY425883... at 55.52%
4	678aa, >KY425901... at 53.39%
5	609aa, >KY425965... at 51.23%
6	609aa, >KY425968... at 51.40%
7	909aa, >KY425869... at 55.01%
8	744aa, >KY425892... at 55.78%
9	801aa, >KY425888... at 55.56%
10	513aa, >KY425914... at 50.49%
11	621aa, >KY425863... at 55.72%
12	855aa, >KY425880... at 54.74%
13	579aa, >KY426003... at 51.12%
14	450aa, >KY425915... at 52.89%
15	450aa, >KY425916... at 51.56%
16	879aa, >KY425840... at 56.88%
17	909aa, >KY425870... at 55.67%
18	849aa, >KY425887... at 55.01%
19	555aa, >KY425999... at 51.71%
20	414aa, >KY425933... at 50.72%
21	621aa, >KY425862... at 56.20%
22	450aa, >KY425917... at 52.89%
23	678aa, >KY425878... at 53.39%
24	843aa, >KY425825... at 55.16%
25	555aa, >KY425998... at 50.09%
26	669aa, >KY425992... at 51.72%
27	849aa, >KY425886... at 55.24%
28	912aa, >KY425836... at 52.52%
29	858aa, >KY4